In [ ]:
import numpy as np
import cvxpy as cp
import cvxopt
import matplotlib.pyplot as plt
from AnsatzLacs import *
from PurificationUtil import *
from qiskit_algorithms.optimizers import SPSA
import sys

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
total_size = sys_size
num_layers = 3

In [ ]:
# define hamiltonian 
h_coefs = [1]
h_paulis = ['ZZ']

# define the constraints
a_coefs = [[.5],[.7]]
a_paulis = [['ZI'],['IZ']]

# define the constraint values
b_vals = [.1,.3]

In [ ]:
def operator_dot_product(a_coefs, a_paulis, b_coefs, b_paulis):
    prod = 0
    for i in range(len(a_coefs)):
        for j in range(len(b_coefs)):
            if a_paulis[i] == b_paulis[j]:
                prod += a_coefs[i] * b_coefs[j]
    return 2**sys_size * prod

In [ ]:
# precompute terms
h_2 = 2**sys_size*np.sum(np.square(h_coefs))
h_I = 0 if 'I'*sys_size not in h_paulis else 2**sys_size*h_coefs[h_paulis.index('I'*sys_size)]

h_a = []
for i in range(len(a_coefs)):
    dot_product = operator_dot_product(h_coefs,h_paulis,a_coefs[i],a_paulis[i])
    h_a.append(dot_product)

a_I = []
for i in range(len(a_coefs)):
    a_I.append(0 if 'I'*sys_size not in a_paulis else 2**sys_size*a_coefs[a_paulis.index('I'*sys_size)])

a_i_a_j = np.zeros((len(a_coefs),len(a_coefs)))
for i in range(len(a_coefs)):
    for j in range(len(a_coefs)):
        a_i_a_j[i,j] = operator_dot_product(a_coefs[i],a_paulis[i],a_coefs[j],a_paulis[j])

a_2 = []
for i in range(len(a_coefs)):
    a_2.append(2**sys_size*np.sum(np.square(a_coefs[i])))

In [ ]:
def get_pauli(pauli_char):
    if pauli_char == 'I':
        return np.array([1,1])
    elif pauli_char == 'Z':
        return np.array([1,-1])

def get_pauli_prod(pauli_string):
    pauli = np.ones(1)
    for char in pauli_string[::-1]:
        pauli = np.kron(pauli,get_pauli(char))
    return pauli

def hamiltonian(coefs,paulis):
    matrix = np.zeros(2**sys_size)
    for i in range(len(coefs)):
        matrix += coefs[i] * get_pauli_prod(paulis[i])
    return matrix

# use SDP to calculate the value
d = 2**sys_size
I = cvxopt.matrix(np.ones(d))
H = cvxopt.matrix(hamiltonian(h_coefs,h_paulis))
P = cp.Variable(d)
Y = cp.Variable(len(a_coefs))
MU = cp.Variable(1)

constraints = [P >= 0, cp.sum(P) == 1, Y >= 0]

sum_constraint = MU*I
objective = MU
for i in range(len(a_coefs)):
    sum_constraint += Y[i] * cvxopt.matrix(hamiltonian(a_coefs[i],a_paulis[i]))
    objective += b_vals[i]*Y[i]
constraints.append(sum_constraint <= H)

prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()
expected = prob.value
print("Estimated Value:", round(prob.value,5))

In [ ]:
def get_exp(pauli_string, ansatz, shots):
    probs = ansatz.get_prob(shots)
    if shots is None:
        return np.dot(probs,get_pauli_prod(pauli_string))

    else:
        rng = np.random.default_rng()
        counts = rng.multinomial(shots, probs)
        probs = counts / shots

        return np.dot(probs,get_pauli_prod(pauli_string))
    
def get_trace(ansatz1, ansatz2, shots):
    '''
    Performs a descructive swap test (bell measurement) and expectation value calculation to produce the
    trace of the inner product of the state described by params1 and the state described by params2.
    To evaluate Tr[AB], ansatz1 should prepare state A and ansatz2 should prepare state B
    
    param shots: the number of shots executed on the quantum circuit over which to average
    return: floating point value for the trace
    '''
    probs1 = ansatz1.get_prob(shots)
    probs2 = ansatz2.get_prob(shots)
    return np.dot(probs1,probs2)

In [ ]:
# customized SPSA optimizer
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation * delta)
    minus = fun(params - perturbation * delta)
    grad = (plus - minus) / (2 * perturbation * delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Primal Optimization

In [ ]:
def cost_function_full_primal(params):
    """
    Returns the value of the cost function
    """
    l_val = len(b_vals)
    
    param = params[:num_parameters]
    p_ansatz.set_params(param)
    y = np.array(params[num_parameters:num_parameters + l_val])**2
    mu = params[-2]
    lmbda = (params[-1])**2
    
    # evaluate the objective function
    obj = np.dot(b_vals,y) + mu

    p1 = h_2
    p2 = y.T @ a_i_a_j @ y
    p3 = mu**2 * 2**sys_size
    p4 = lmbda**2 * get_trace(p_ansatz,p_ansatz,num_shots)
    p5 = -np.dot(y,h_a)
    p6 = mu * h_I
    p7 = 0
    
    for i in range(len(h_coefs)):
        p7 -= lmbda * h_coefs[i] * get_exp(h_paulis[i],p_ansatz,num_shots)
    
    a_z = []
    for i in range(len(a_coefs)):
        a_zi = 0
        for j in range(len(a_coefs[i])):
            a_zi += lmbda * a_coefs[i][j] * get_exp(a_paulis[i][j],p_ansatz,num_shots)
        a_z.append(a_zi)
    
    p8 = np.dot(y,a_z)
    p9 = mu * np.dot(y,a_I)
    p10 = mu * lmbda
    
    penalty = p1 + p2 + p3 + p4 + 2 * (p5 + p6 + p7 + p8 + p9 + p10)
    
    cost = obj - c * penalty
    
    return cost, obj, penalty

def cost_function_primal(params):
    cost, obj, penalty = cost_function_full_primal(params)
    return -cost

In [ ]:
print_every = 100
def callback_primal(x, iteration):
    global lr
    global lr_wait
    
    fx, obj, penal = cost_function_full_primal(x)

    primal_iterations.append(iteration)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
    
    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 3:
            m,_ = np.polyfit(primal_iterations[-300:], primal_loss[-300:], 1)
            if m < 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
                
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
max_iter = 8000
lr_max = 1e-1
lr_min = 1e-2
perturbation = 1e-4
num_train = 10

In [ ]:
num_train = 3
record = 0

iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    c = 10
    
    np.random.seed(i)
    p_ansatz = create_ansatz('qcbm', sys_size, 0, num_layers)
    num_parameters = p_ansatz.get_parameter_num()

    print("Expected value:", expected)
    init_params = [*p_ansatz.params, *np.zeros(len(b_vals)), 0, 1]
    init_cost, init_obj, init_penal = cost_function_full_primal(init_params)
    
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objectives = [init_obj]
    primal_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(cost_function_primal, init_params, get_gradient, max_iter, callback=callback_primal)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)
    iterations_primal_outer.append(primal_iterations)
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"CH_CSlack_primal_data"
        problem_name = "CH"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Dual Optimization

In [ ]:
def cost_function_full_dual(params):
    """
    Returns the value of the cost function
    """
    l_val = len(b_vals)
    
    param = params[:num_parameters]
    z = np.array(params[-l_val:])**2
    p_ansatz.set_params(param)
    
    tr_vec = []
    for i in range(len(h_coefs)):
        tr = get_exp(h_paulis[i], p_ansatz, num_shots)
        tr_vec.append(tr)
    obj = np.dot(h_coefs,tr_vec)
    
    penalty = 0
    for i in range(l_val):
        tr_vec = []
        for j in range(len(a_coefs[i])):
            tr = get_exp(a_paulis[i][j], p_ansatz, num_shots)
            tr_vec.append(tr)
        penalty += (np.dot(a_coefs[i],tr_vec) - b_vals[i] - z[i])**2
    
    cost = obj + c * penalty
    
    return cost, obj, penalty

In [ ]:
def cost_function_dual(params):
    cost, obj, penalty = cost_function_full_dual(params)
    return cost

In [ ]:
print_every = 100
def callback_dual(x, iteration):
    global lr
    global lr_wait
    
    fx, obj, penal = cost_function_full_dual(x)

    dual_iterations.append(iteration)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)
    
    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 3:
            m,_ = np.polyfit(dual_iterations[-300:], dual_loss[-300:], 1)
            if m > 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
                
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
max_iter = 8000
lr_max = 1e-1
lr_min = 1e-2
perturbation = 1e-4
num_train = 10

In [ ]:
num_train = 3
record = 0

iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    c = 100
    
    np.random.seed(i)
    p_ansatz = create_ansatz('qcbm', sys_size, 0, num_layers)
    num_parameters = p_ansatz.get_parameter_num()

    print("Expected value:", expected)
    init_params = [*p_ansatz.params, *np.zeros(len(b_vals))]
    init_cost, init_obj, init_penal = cost_function_full_dual(init_params)
    
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objectives = [init_obj]
    dual_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(cost_function_dual, init_params, get_gradient, max_iter, callback=callback_dual)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)
    iterations_dual_outer.append(dual_iterations)
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"CH_CSlack_dual_data"
        problem_name = "CH"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 100
window_size_dual = 100

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
plt.ylim(-2, 1)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()